# Text 1: Vector space models
**Internet Analytics - Lab 4**

---

**Group:** *Your group letter.*

**Names:**

* *Name 1*
* *Name 2*
* *Name 3*

---

#### Instructions

*This is a template for part 1 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [ ]:
import pickle
import numpy as np
from scipy.sparse import csr_matrix
from utils import load_json, load_pkl
import string
import math
import collections
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

courses = load_json('data/courses.txt')
stopwords = load_pkl('data/stopwords.pkl')


## Exercise 4.1: Pre-processing

In [ ]:
newCourses =[]
ps = PorterStemmer()
translator = str.maketrans('', '', string.punctuation)
for course in courses:
    temp = course['description'].lower()
    for word in stopwords:
        temp = temp.replace(' '+word+' ',' ')
    temp = temp.translate(translator)
    stemList = []
    #for word in temp.split(' '):
    #    stemList.append(ps.stem(word))
    #temp = ' '.join(stemList)
        
    newCourses.append({'name':course['name'],'description':temp.split(' ')})

In [ ]:
def countDocWithTerm(term,docs):
    result = 0
    for doc in docs:
        if term in doc['description']:
            result += 1
    return result
            

terms =[]
for item in newCourses:
    terms.extend(item['description'])
terms = list(collections.Counter(terms).keys())
terms.remove('')
termsDict ={}
for i,term in enumerate(terms):
    termsDict[i]=term
termsDict = dict(collections.OrderedDict(sorted(termsDict.items())))
print(termsDict)
newCoursesDict = {}
for i,doc in enumerate(newCourses):
    newCoursesDict[i]=doc
newCoursesDict = dict(collections.OrderedDict(sorted(newCoursesDict.items())))
values =[]
rows=[]
columns=[]
print(len(terms))
for i,term in termsDict.items():
    if i%250 == 0:
        print(i)
    for index,doc in newCoursesDict.items():
        if term in doc['description']:
            tf = doc['description'].count(term)/len(doc['description'])
            idf = math.log(len(newCourses)/countDocWithTerm(term,newCourses))
            tf_idf=tf*idf
            values.append(tf_idf)
            rows.append(i)
            columns.append(index)
X = csr_matrix((values, (rows, columns)), shape=(len(terms), len(newCourses)))
print(X)
            
            

## Exercise 4.2: Term-document matrix

In [ ]:
#X = X.toarray()
a = 0
for key,value in newCoursesDict.items():
    if value['name'] == 'Internet analytics':
        a = key
        break;
temp =np.argsort(X[:,43])[::-1][:15]
for i in temp:
    print(termsDict[i])

## Exercise 4.3: Document similarity search